In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import chromedriver_binary
from io import BytesIO
import base64
from python3_anticaptcha import ImageToTextTask, CallbackClient
from PIL import Image
from time import sleep
import requests
import time
import config as cg

idCard = cg.IDCardInfo()
anticaptcha = cg.AntiCaptchaInfo()


# 前臺開啟瀏覽器模式
def openChrome():
    # 加啟動配置
    option = webdriver.ChromeOptions()
    option.add_argument('disable-infobars')
    option.add_argument("--window-size=1280,1024")
    # 開啟chrome瀏覽器
    driver = webdriver.Chrome(chrome_options=option)
    return driver

def parse_captcha(image_path):
    # Load image into memory
    buffer = BytesIO()
    image = Image.open(image_path)
    image.save(buffer, format="png")
    # Use base64 to encode image buffer
    img_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    # Anti-captcha API structure
    data = {
        "clientKey":anticaptcha.key,
        "task": {
            "type": "ImageToTextTask",
            "body": img_str,
            "phrase":False,
            "case": True,
            "numeric": 0,
            "math": 0,
            "minLength": 5,
            "maxLength": 5
        }
    }
    # Create a ImageToTextTask and retrieve taskId from response
    r = requests.post("https://api.anti-captcha.com/createTask", json=data)
    r.raise_for_status()
    task_id = r.json()['taskId']
    # Polling for task finish.
    ret = ""
    while True:
        data = {
            "clientKey":anticaptcha.key,
            'taskId': task_id
        }
        r = requests.post("https://api.anti-captcha.com/getTaskResult", json=data)
        r.raise_for_status()
        if r.json()['status'] == 'ready':
            ret = r.json()['solution']['text']
            break
        time.sleep(3)
    return ret

In [ ]:
#collect images
driver = openChrome()

url = "https://www.ris.gov.tw/app/portal/3043"
driver.get(url)
driver.implicitly_wait(15)
driver.switch_to.frame("content-frame")

count = 0
while (count<7):
    count += 1
    path = "./images/"+str(count)+'.png'
    
    driver.find_element_by_id("captchaInput_captcha-refresh").send_keys("")
    
    get_captcha(driver, path)
    time.sleep(1)
    
    driver.set_window_size(1280, 1024)
    
    driver.find_element_by_id("imageBlock_captcha-refresh").click()

In [ ]:
#use anti-captcha to label 
import os

for i in range(2448, 2988):
    file_path = "./images/" + str(i) + '.png'
    new_name = "./images/"+ parse_captcha(file_path)+'.png'
    os.rename(file_path, new_name)
    

In [ ]:
#anti-captcha result manual correction
import os
folder = "./images_bw"
for filename in os.listdir(folder):
    if len(filename) == 9:
        new_name = filename[:5].upper() + '.png'
        os.rename(os.path.join(folder,filename), os.path.join(folder,new_name))


In [ ]:
#collect label name into one-hot encoding and turn into csv file, rename images with order as well
import os
import numpy as np

folder = "./images_bw"
count = 0
dic = {'2':0, '3':1, '4':2, '5':3,'6':4, '7':5, '8':6, '9':7, 
         'A': 8, 'B':9,'C':10, 'D':11, 'E':12, 'F':13, 'G':14, 'H':15,
         'J':16, 'K':17, 'N':18, 'P':19,'Q':20, 'R':21, 'S':22, 'T':23,
         'U':24, 'V':25, 'X':26, 'Y':27, 'Z':28}

label_file = np.zeros(shape=(3000, 5, 29))

for old_filename in os.listdir(folder):
    new_filename = str(count)+'.png'
    
    label = old_filename[:5]
    
    for i in range(len(label)):
        if label[i] in dic:
            x = dic.get(label[i])
            label_file[count][i][x] = 1
        else:
            print('label not exist ='+label)
        
    
    os.rename(os.path.join(folder,old_filename), os.path.join(folder,new_filename))
    count+=1

import pandas as pd
label_file = label_file.reshape(3000, 5*29)
pd.DataFrame(label_file).to_csv("./label_file.csv", index=False)

In [ ]:
import cv2
import os
folder = "./images"
bw_folder = './images_bw'
for filename in os.listdir(folder):
    #print(os.path.join(folder,filename))
    originalImage = cv2.imread(os.path.join(folder,filename))
    grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
    (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 130, 255, cv2.THRESH_BINARY)
    
    cv2.imwrite(os.path.join(bw_folder,filename), blackAndWhiteImage)